<img src="img/saturn_logo.png" width="300" />

# Set Up Training

We don't need to run all of Notebook 5 again, we'll just call `setup2.py` in the next chunk to get ourselves back to the right state.

In [1]:
%run -i setup2.py

[2020-12-03 22:45:06] INFO - dask-saturn | Cluster is ready
[2020-12-03 22:45:06] INFO - dask-saturn | Registering default plugins
[2020-12-03 22:45:06] INFO - dask-saturn | {'tcp://10.0.0.150:35343': {'status': 'repeat'}, 'tcp://10.0.24.234:35189': {'status': 'repeat'}, 'tcp://10.0.3.113:42823': {'status': 'repeat'}}


In [2]:
client

Client Scheduler: tcp://d-steph-resnet-article-50680500b0454380ace50ab2e594ca29.main-namespace:8786 Dashboard: https://d-steph-resnet-article-50680500b0454380ace50ab2e594ca29.internal.saturnenterprise.io,Cluster Workers: 3 Cores: 24 Memory: 181.50 GB


We're ready to do some learning! 

## Regular Model Details

Aside from the Special Elements noted below, we can write this section essentially the same way we write any other PyTorch training loop. 
* Cross Entropy Loss for our loss function
* SGD (Stochastic Gradient Descent) for our optimizer

We're also using a particular learning rate scheduler called `ReduceLROnPlateau` which leaves our base learning rate alone until the model's efforts hit a plateau and the loss function is no longer decreasing.

We have two stages in this process, as well - training and evaluation. We run the training set completely using batches of 100 before we move to the evaluation step, where we run the eval set completely also using batches of 100.

***

## Special Elements

Most of the training workflow function shown below is pretty standard for users of PyTorch. However, there are a couple of elements that are different.

### DaskResultsHandler
In order to use the model output handler, we need to initialize the `DaskResultsHandler` class for our experiment, from `dask-pytorch-ddp`.
This object has a few important methods, including letting our model performance at each iteration be automatically documented.  

In [3]:
import uuid
key = uuid.uuid4().hex

rh = results.DaskResultsHandler(key)


### Worker Rank
```
worker_rank = int(dist.get_rank())
```

This is checking to see which of the workers in the cluster we're on. This way, our results records can tell which worker this performance represents.


### Model to Device

```
device = torch.device(0)
net = models.resnet50(pretrained=True)
model = net.to(device)
```

As you'll recall from Notebook 4, we need to make sure our model is placed on the worker- here we do it one time before the training loops begin. We will also pass each image and its label to the worker within the training and evaluation loops - see if you can find this spot, you need to fill in the blanks!


### DDP Wrap
```
device_ids = [0]
model = DDP(model, device_ids=device_ids)
```

And finally, we need to enable the DistributedDataParallel framework. To do this, we are using the `DDP()` wrapper around the model, which is short for the PyTorch function `torch.nn.parallel.DistributedDataParallel`. There is a lot to know about this, but for our purposes the important thing is to understand that this allows the model training to run in parallel on our cluster. https://pytorch.org/docs/stable/notes/ddp.html
***

## Discussing DDP
It's helpful to know what this framework is really doing under the hood.

The official PyTorch documentation tells us this:

```This container parallelizes the application of the given module by splitting the input across the specified devices by chunking in the batch dimension (other objects will be copied once per device). In the forward pass, the module is replicated on each device, and each replica handles a portion of the input. During the backwards pass, gradients from each replica are summed into the original module.```

Clear as mud, right? Let’s try to break it down.

`This container parallelizes the application of the given module`

This is just indicating that we’re parallelizing a deep learning workflow — transfer learning in our case.

`by splitting the input across the specified devices by chunking in the batch dimension`

Input for a transfer learning workflow is the dataset! Ok, so it is chunking our image batches and that’s what gets to be parallel.

`(other objects will be copied once per device)`

Eg, our starting model, if any (Resnet50 for us) doesn’t get broken up at all. Good to know.

`In the forward pass, the module is replicated on each device, and each replica handles a portion of the input.`

Ok, so the training task, our module, is replicated on each device. We have multiple copies of the job working simultaneously, and each one gets a chunk of the input images rather than the entire dataset.

`During the backwards pass, gradients from each replica are summed into the original module.`

And then each of these duplicate tasks passes the results (the gradients) back home to the original! The learning is happening out in the workers/child processes, and then they all return the results to the original module/training process to be aggregated.

The essential difference with DDP, then, is that it is optimized for multiple machines instead of a single machine with multiple threads. It’s able to communicate across difference machines effectively, so we can use a GPU cluster for our computation.

For a more detailed discussion and more tips about this same workflow, [you can visit our blog to read more!](https://www.saturncloud.io/s/combining-dask-and-pytorch-for-better-faster-transfer-learning/)

***

## Train that Baby!
Our whole training process is going to be contained in one function, here named `run_transfer_learning`. There are some empty spots here referring to concepts we have discussed. Fill in the blanks in between `<<< >>>` marks to get the correct training function, or click the ellipsis below to check your work.

In [4]:
import torch
from torch import nn, optim
from torch.nn.parallel import DistributedDataParallel as DDP

from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import torch.distributed as dist
from torch.optim import lr_scheduler

In [ ]:
### FILL IN THE BLANKS ###

def run_transfer_learning(bucket, prefix, train_pct, batch_size, n_epochs, base_lr):
    '''Load basic Resnet50, run transfer learning over given epochs.
    Uses dataset from the path given as the pool from which to take the 
    training and evaluation samples.'''
    
    # --------- Format model and params --------- #
    worker_rank = int(dist.get_rank())
    
    device = torch.device(0)
    net = models.resnet50(pretrained=True)
    model = net.to(<<< FILL IN >>>)
    device_ids = [0]
    model = <<< FILL IN >>>(model, device_ids=device_ids)
    
    criterion = nn.CrossEntropyLoss().cuda()    
    lr = base_lr * dist.get_world_size()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'min', patience = 2)
    
    # --------- Retrieve data for training and eval --------- #
    whole_dataset = prepro_batches(bucket, prefix)
    train, val = get_splits_parallel(train_pct, whole_dataset, batch_size=batch_size)
    dataloaders = {'train' : train, 'val': val}

    # --------- Start iterations --------- #
    count = 0
    t_count = 0
    for epoch in range(n_epochs):
    # --------- Training section --------- #    
        model.train()  
        for inputs, labels in dataloaders["train"]:
            dt = datetime.datetime.now().isoformat()
            inputs = inputs.to(device)
            labels = labels.to(<<< FILL IN >>>)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            correct = (preds == labels).sum().item()
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            count += 1

            for param_group in optimizer.param_groups:
                current_lr = param_group['lr']
                
            rh.submit_result(
                f"worker/{worker_rank}/data-{dt}.json", 
                json.dumps({'loss': loss.item(),'learning_rate':current_lr, 'correct':correct, 'epoch': epoch, 'count': count, 'worker': worker_rank, 'sample': 'train'})
            )
        
            if (count % 100) == 0 and worker_rank == 0:
                rh.submit_result(f"checkpoint-{dt}.pkl", pickle.dumps(model.state_dict()))

    # --------- Evaluation section --------- #   
        with torch.no_grad(): 
            model.eval()
            for inputs_t, labels_t in dataloaders["val"]:
                dt = datetime.datetime.now().isoformat()
                inputs_t = inputs_t.to(<<< FILL IN >>>)
                labels_t = labels_t.to(device)
            
                outputs_t = model(inputs_t)
                _,pred_t = torch.max(outputs_t, dim=1)
                loss_t = criterion(outputs_t, labels_t)
                correct_t = (pred_t == labels_t).sum().item()
                t_count += 1

                for param_group in optimizer.param_groups:
                    current_lr = param_group['lr']
                    
                rh.submit_result(
                    f"worker/{worker_rank}/data-{dt}.json", 
                    json.dumps({'loss': loss_t.item(),'learning_rate':current_lr, 'correct':correct_t, 'epoch': epoch, 'count': t_count, 'worker': worker_rank, 'sample': 'eval'})
                )

        scheduler.step(loss)

In [6]:
def run_transfer_learning(bucket, prefix, train_pct, batch_size, n_epochs, base_lr):
    '''Load basic Resnet50, run transfer learning over given epochs.
    Uses dataset from the path given as the pool from which to take the 
    training and evaluation samples.'''
    # --------- Format model and params --------- #
    worker_rank = int(dist.get_rank())
    
    device = torch.device(0)
    net = models.resnet50(pretrained=True)
    model = net.to(device)
    device_ids = [0]
    model = DDP(model, device_ids=device_ids)
    
    criterion = nn.CrossEntropyLoss().cuda()    
    lr = base_lr * dist.get_world_size()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'min', patience = 2)
    
    # --------- Retrieve data for training and eval --------- #
    whole_dataset = prepro_batches(bucket, prefix)
    train, val = get_splits_parallel(train_pct, whole_dataset, batch_size=batch_size)
    dataloaders = {'train' : train, 'val': val}

    # --------- Start iterations --------- #
    count = 0
    t_count = 0
    for epoch in range(n_epochs):
    # --------- Training section --------- #    
        model.train()  # Set model to training mode
        for inputs, labels in dataloaders["train"]:
            dt = datetime.datetime.now().isoformat()
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            correct = (preds == labels).sum().item()
            
            # zero the parameter gradients
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            count += 1
            # statistics
            for param_group in optimizer.param_groups:
                current_lr = param_group['lr']
            # Record the results of this model iteration (training sample) for later review.
            rh.submit_result(
                f"worker/{worker_rank}/data-{dt}.json", 
                json.dumps({'loss': loss.item(),'learning_rate':current_lr, 'correct':correct, 'epoch': epoch, 'count': count, 'worker': worker_rank, 'sample': 'train'})
            )
        
            if (count % 100) == 0 and worker_rank == 0:
                # Grab a snapshot of the current state of the model, in case of interruption or need to review
                rh.submit_result(f"checkpoint-{dt}.pkl", pickle.dumps(model.state_dict()))

    # --------- Evaluation section --------- #   
        with torch.no_grad():
            model.eval()  # Set model to evaluation mode
            for inputs_t, labels_t in dataloaders["val"]:
                dt = datetime.datetime.now().isoformat()
                inputs_t = inputs_t.to(device)
                labels_t = labels_t.to(device)
            
                outputs_t = model(inputs_t)
                _,pred_t = torch.max(outputs_t, dim=1)
                loss_t = criterion(outputs_t, labels_t)
                correct_t = (pred_t == labels_t).sum().item()
                t_count += 1

                # statistics
                for param_group in optimizer.param_groups:
                    current_lr = param_group['lr']
                # Record the results of this model iteration (evaluation sample) for later review.
                rh.submit_result(
                    f"worker/{worker_rank}/data-{dt}.json", 
                    json.dumps({'loss': loss_t.item(),'learning_rate':current_lr, 'correct':correct_t, 'epoch': epoch, 'count': t_count, 'worker': worker_rank, 'sample': 'eval'})
                )

        scheduler.step(loss)

Now we've done all the hard work, and just need to run our function! Using `dispatch.run` from `dask-pytorch-ddp`, we pass in the transfer learning function so that it gets distributed correctly across our cluster. This creates futures and starts computing them, while we use `process_results` to return the performance and learning statistics. 

### Define Model Parameters

As with any PyTorch model, you'll want to define the epochs of training you plan to do, the batch size if using batches, and the starting learning rate. We're also able to assign the train/test split here because of how the functions above are written.

(We're using only two epochs here to save time, but of course you can increase this.)

In [7]:
startparams = {'n_epochs': 2, 
               'batch_size': 100,
               'train_pct': .8,
               'base_lr': 0.01}

In [8]:
import math
import numpy as np
import multiprocessing as mp
import datetime
import json 
import pickle

num_workers = 64

### Send Tasks to Workers
 
We talked in Notebook 2 about how we distribute tasks to the workers in our cluster, and now you get to see it firsthand. Inside the `dispatch.run()` function in `dask-pytorch-ddp`, we are actually using the `client.submit()` method to pass tasks to our workers, and collecting these as futures in a list. We can prove this by looking at the results, here named "futures", where we can see they are in fact all pending futures, one for each of the workers in our cluster.

Remember, each worker is attacking the same problem - our transfer learning problem - and pursuing a solution simultaneously. We'll have one and only one job per worker.

In [9]:
%%time    
futures = dispatch.run(client, run_transfer_learning, bucket = "saturn-public-data", prefix = "dogs/Images", **startparams)
futures

CPU times: user 26.6 ms, sys: 0 ns, total: 26.6 ms
Wall time: 25.6 ms


[<Future: pending, key: dispatch_with_ddp-9e0964d3c0f27c2de7be58c9eb7e9a38>,
 <Future: pending, key: dispatch_with_ddp-e559c9da599c666b647fc482d3d58282>,
 <Future: pending, key: dispatch_with_ddp-1da294f872ab79b9c84bbb23e2ed0468>]

In [10]:
display(HTML(gpu_links))

<img src="https://media.giphy.com/media/VFDeGtRSHswfe/giphy.gif" alt="parallel" style="width: 200px;"/>

Now we let our workers run for awhile. This step will take time, so you may not be able to see the full results during our workshop. (In tests, it took about 13 minutes to do two epochs.) See the two links above to view the GPUs efforts as the job runs. This won't hold up your Jupyter environment here, but I promise the cluster is working hard!

### Retrieve Results

This step is where we gather up and save the results. While the cluster is working away at the computation, we can run the `process_results()` method on the DaskResultsHandler. This will be us requesting the results of each future as they run, which is familiar from [Notebook 4](04-parallel-inference.ipynb), where we used `fut.result()`. To see partial results coming in, you should have the `workshop_results` folder in the folder menu a few moments after you run the next two chunks. Look in this folder to see the results each worker is returning to us.

In [11]:
!rm -rf /home/jovyan/project/workshop-dask-pytorch/workshop_results

In [ ]:
%%time

rh.process_results("/home/jovyan/project/workshop-dask-pytorch/workshop_results", futures, raise_errors=False)

This task will continue to hold up your Jupyter instance until it has been able to collect all the results.

## Proof of Results

We don't have the time today to run an assortment of different cluster sizes to see what works best, but I happen to have the results of those runs saved and visualized, to demonstrate how well it works! [Follow me to Notebook 7!](07-learning-results.ipynb)